In [ ]:
# default_exp data.scalers

# Scalers 
> Utils for scaling data

In [ ]:
# export
import torch as t
# import statsmodels.api as sm

In [ ]:
# export
class Scaler(object):
    def __init__(self, technique):
        """ Scale features by shifting their center and scaling.

        The Scaler class contains methods to preprocess features, centering them
        and bounding their variance. Centering and scaling are computed independently 
        on each series' feature. 

            Parameters:
                technique: str
                    The scaling technique is defined by the scale `technique` parameter that 
                    takes one  of the following values: 
                    ['Std', 'Median', 'Norm', 'Norm1', 'Invariant']

            Attributes:
                center_: array of shape (n_series, n_features) or None
                    Per feature/series center (mean/median) of the data to center around zero.

                scale_: array of shape (n_series, n_features, ) or None
                    Per feature/series scaling of the data to bound variance.

            References:
            [1] LeCun, Y., Bottou, L., Orr, G. B., & Müller, K. R. (1998). Effcient backprop. 
                In Neural Networks: Tricks of the Trade pp. 9{50). Berlin, Heidelberg: Springer 
                Berlin Heidelberg. URL: https://link.springer.com/chapter/10.1007/3-540-49430-8_2.

            [2] Kuhn, M., & Johnson, K. (2019). Feature engineering and selection: A practical 
                approach for predictive models. CRC Press.
            
        """

        assert (technique in ['std']), 'Technique not defined'
        self.technique = technique
        self.center_ = None
        self.scale_ = None
        self.fitted = False

    def fit(self, X, mask):
        """ Scale features of X according to the scale `technique`.


        Parameters
        ----------
            X: array-like of shape (n_series, n_features, n_time)
                The input data to transform.

            mask: array-like of shape (n_series, n_time)
                Mask on which to compute the center_ and scale_. You can use it to 
                separate available and in-sample and out-sample data.
                So that test set leakage is avoided.

        """
        if mask is None:
            mask = t.ones((X.shape))
        else:
            mask = mask[:, None, :]

        if self.technique == 'std':
            self.center_, self.scale_ = fit_std_scaler(X, mask)

        self.fitted = True

        return self

    def transform(self, X):
        assert self.fitted, 'Scaler must be fitted before transforming'

        if self.technique == 'std':
            X_scaled = transform_std_scaler(X, center=self.center_, scale=self.scale_)
        
        nan_before_scale = t.sum(t.isnan(X))
        nan_after_scale = t.sum(t.isnan(X_scaled))

        assert nan_before_scale == nan_after_scale, 'Scaler induced nans'

        return X_scaled

    def inverse_transform_y(self, Y, ts_idxs):
        """ Return inverse-transformed input Y, defined by the scale `technique`.

        Parameters
        ----------
            Y: array-like of shape (n_windows, n_time)
                The input data to transform.
            ts_idxs: array-like of shape (n_windows)
                Time-series index for each n_window in Y.
                Indexes must be within [0, n_series)

        Returns
        -------
            Y_inv_scaled: array-like of shape (n_windows, n_time)
                Transformed data.
        """
        assert self.fitted, 'Scaler must be fitted before transforming'

        # 0 index target variable in center and
        # scale (inherited from ts_tensor in Dataset object)
        center = self.center_[ts_idxs, 0]
        scale  = self.scale_[ts_idxs, 0]

        if self.technique == 'std':
            Y_inv_scaled = inv_std_scaler(X=Y,
                                          center=center, 
                                          scale=scale)

        return Y_inv_scaled

# Standard scaler
def fit_std_scaler(X, mask):
    """
    Standardize features by removing the mean and scaling to unit variance.

    The scaled features are computed as:

    $$ \tilde{X} = (X-\mu) / \sigma $$

    """
    means = t.sum(X*mask, dim=-1, keepdim=True) / t.sum(mask, dim=-1, keepdim=True)
    stds = t.sqrt( t.sum( mask*(X-means)**2, dim=-1, keepdim=True)/ \
                    t.sum(mask, dim=-1, keepdim=True) )

    return means, stds

def transform_std_scaler(X, center, scale):
    return (X - center) / scale

def inv_std_scaler(X, center, scale):
    """
    Scale back the data to the original representation.

    The scaled features are computed as:

    $$ X = \tilde{X} * \sigma +\mu $$

    """
    return (X * scale) + center

# # Norm
# def norm_scaler(x, mask):    
#     x_max = np.max(x[mask==1])
#     x_min = np.min(x[mask==1])
    
#     x = (x - x_min) / (x_max - x_min)
#     return x, x_min, x_max

# def inv_norm_scaler(x, x_min, x_max):
#     return x * (x_max - x_min) + x_min

# # Norm1
# def norm1_scaler(x, mask):
#     x_max = np.max(x[mask==1])
#     x_min = np.min(x[mask==1])

#     x = (x - x_min) / (x_max - x_min)
#     x = x * (2) - 1
#     return x, x_min, x_max

# def inv_norm1_scaler(x, x_min, x_max):
#     x = (x + 1) / 2
#     return x * (x_max - x_min) + x_min

# # Median
# def median_scaler(x, mask):
#     x_median = np.median(x[mask==1])
#     x_mad = sm.robust.scale.mad(x[mask==1])
#     if x_mad == 0:
#         x_mad = np.std(x[mask==1], ddof = 1) / 0.6744897501960817
#     x = (x - x_median) / x_mad
#     return x, x_median, x_mad

# def inv_median_scaler(x, x_median, x_mad):
#     return x * x_mad + x_median

# # Invariant
# def invariant_scaler(x, mask):
#     x_median = np.median(x[mask==1])
#     x_mad = sm.robust.scale.mad(x[mask==1])
#     if x_mad == 0:
#         x_mad = np.std(x[mask==1], ddof = 1) / 0.6744897501960817
#     x = np.arcsinh((x - x_median) / x_mad)
#     return x, x_median, x_mad

# def inv_invariant_scaler(x, x_median, x_mad):
#     return np.sinh(x) * x_mad + x_median

# Usage Example

In [ ]:
import unittest
import numpy as np
import torch as t

In [ ]:
class TestScaler(unittest.TestCase):
    def __init__(self):
        t.manual_seed(1)
        X_1 = t.randn((7,4,10000))
        X_2 = 5*t.randn((7,4,10000)) + 3
        self.X = t.cat((X_1, X_2), dim=2)
        self.mask = t.ones((7,20000))
        self.mask[:,-10000:] = 0
        self.scaler = Scaler(technique='std')

    def test_std(self):
        self.scaler = self.scaler.fit(X=self.X, mask=self.mask)
        X_scaled = self.scaler.transform(self.X)
        
        np.testing.assert_almost_equal(self.scaler.center_.numpy(), np.zeros(self.scaler.center_.shape), decimal=1)
        np.testing.assert_almost_equal(self.scaler.scale_.numpy(), np.ones(self.scaler.center_.shape), decimal=1)
        np.testing.assert_almost_equal(self.X.numpy(), X_scaled.numpy(), decimal=0)

In [ ]:
test = TestScaler()
test.test_std()

In [ ]:
test.scaler.center_.shape

In [ ]:
test.scaler.center_[:,0].shape